In [1]:
abstract type Abstractgrid end
abstract type Abstractgridfunction end

######################## Eulerian grid ######################
#############################################################

struct PeriodicEulergrid <: Abstractgrid
    Lx1::Float64   #Initial point in row direction on the grid
    Lx2::Float64   #End point in row direction on the grid
    Uy1::Float64   #Initial point in column direction on the grid
    Uy2::Float64   #End point in column direction on the grid.
    Nx::Int64      #Number of rows on the grid.
    Ny::Int64      #Number of columns on the grid.
    dx::Float64
    dy::Float64
    X::Matrix{Float64} #Row coordinates on the grid i.e constant along rows.
    Y::Matrix{Float64} #Column coordinates on the grid i.e constant along columns.
    L::Float64  #Length of grid
    B::Float64  #Breadth of grid
end
########################Grid construction######################################

function PeriodicEulergrid(Lx1::T,Lx2::T,Uy1::T,Uy2::T,Nx::T,Ny::T) where T<:Real
    L= Lx2-Lx1;
    B=Uy2-Uy1;
    dx=L/Nx;
    dy=B/Ny;
    X=[(i-1/2)*dx+Lx1 for i =1:Nx,j=1:Ny];  #Row data for grid.
    Y=[(j-1/2)*dy + Uy1 for i=1:Nx,j=1:Ny]; #Column data for the grid.
    grid::PeriodicEulergrid = PeriodicEulergrid(Lx1,Lx2,Uy1,Uy2,Nx,Ny,dx,dy,X,Y,L,B)
    return grid
end
#######################Grid construction with some restrictions################

#if Lower limits are not given then assume Lx1=Uy1=0

function PeriodicEulergrid(Lx2::T,Uy2::T,Nx::T,Ny::T) where T<:Real
    Lx1=0.0
    Uy1=0.0
    L= Lx2-Lx1;
    B=Uy2-Uy1;
    dx=L/Nx;
    dy=B/Ny;
    X=[(i-1/2)*dx+Lx1 for i =1:Nx,j=1:Ny];  #Row data for grid.
    Y=[(j-1/2)*dy + Uy1 for i=1:Nx,j=1:Ny]; #Column data for the grid.
    grid::PeriodicEulergrid = PeriodicEulergrid(Lx1,Lx2,Uy1,Uy2,Nx,Ny,dx,dy,X,Y,L,B)
    return grid
end

#if Upper limits are not given then assume Lx2=Uy2=1

function PeriodicEulergrid(Lx1::T,Uy1::T,Nx::T,Ny::T) where T<:Real
    Lx2=1.0
    Uy2=1.0
    L= Lx2-Lx1;
    B=Uy2-Uy1;
    dx=L/Nx;
    dy=B/Ny;
     X=[(i-1/2)*dx+Lx1 for i =1:Nx,j=1:Ny];  #Row data for grid.
    Y=[(j-1/2)*dy + Uy1 for i=1:Nx,j=1:Ny]; #Column data for the grid.
    grid::PeriodicEulergrid = PeriodicEulergrid(Lx1,Lx2,Uy1,Uy2,Nx,Ny,dx,dy,X,Y,L,B)
    return grid
end
#If no data is specified assume Lx1=Uy1=0.0, Lx2=Uy2=1.0
function PeriodicEulergrid(Nx::T,Ny::T) where T<:Real
    Lx2=1.0
    Uy2=1.0
    Uy1=0.0
    Lx1=0.0
    L= Lx2-Lx1;
    B=Uy2-Uy1;
    dx=L/Nx;
    dy=B/Ny;
     X=[(i-1/2)*dx+Lx1 for i =1:Nx,j=1:Ny];  #Row data for grid.
    Y=[(j-1/2)*dy + Uy1 for i=1:Nx,j=1:Ny]; #Column data for the grid.
    grid::PeriodicEulergrid = PeriodicEulergrid(Lx1,Lx2,Uy1,Uy2,Nx,Ny,dx,dy,X,Y,L,B)
    return grid
end

#Now checking the compatibility of data described on our Eulerian grid


#If scalar data is assigned(i.e data has single component at each grid point).
mutable struct coordinatedata <:Abstractgridfunction    
    U::Matrix{Float64}
    grid::PeriodicEulergrid
    function coordinatedata(Ux::Matrix{T},datagrid::PeriodicEulergrid) where T<:Real
        if ~(size(Ux)==size(datagrid.X))
                println("The size is not compatible")
            else
                return new(Ux,datagrid)
            end
        end
    end

# if we have no data specified on Eulerian grid, we will initialize with zeros in both cases

function coordinatedata(datagrid::PeriodicEulergrid)
    Ux=zeros(datagrid.Nx,datagrid.Ny)
    griddata::coordinatedata=coordinatedata(Ux,datagrid)
    return griddata
end

    #This will check for data with multiple components at each grid point.
    mutable struct Vectordata <:Abstractgridfunction    
    U::Matrix{Float64}
    V::Matrix{Float64}
    grid::PeriodicEulergrid
    function Vectordata(U::Matrix{T},V::Matrix{T},datagrid::PeriodicEulergrid) where T<:Real
        if ~(size(U)==size(datagrid.X))
                println("The size is not compatible in first component")
            elseif ~(size(V)==size(datagrid.Y))
                println("The size is not compatible in second component")
                
        else 
                return new(U,V,datagrid)
            end
        end
    end

function Vectordata(datagrid::PeriodicEulergrid)
    Ux=zeros(datagrid.Nx,datagrid.Ny);
    Vy=zeros(datagrid.Nx,datagrid.Ny)
    griddata::Vectordata = Vectordata(Ux,Vy,datagrid)
    return griddata
end
function  Gridintegral(data::Matrix{Float64},mygrid::PeriodicEulergrid)
	if ~(size(data) == (mygrid.Nx,mygrid.Ny))
		println("The size is not compatible")
	end

	summand = @. data*mygrid.dx*mygrid.dy;
	output = sum(summand);
	return output
end

Gridintegral (generic function with 1 method)